In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from itertools import product
from tqdm import tqdm_notebook

path = '../output_data/'

In [ ]:
with pd.HDFStore(path + 'X1a.h5') as store:
    data = store.select('dat_1', where='phi=0.5 & n_rn>15', columns=['decision state', 'i_c', 'G'])
data.index = data.index.droplevel(['phi', 'test'])
data = data.reset_index('tstep')
data = data.reset_index().drop_duplicates(['n_rn', 'n_cp', 'sample', 'tstep']).set_index(['n_rn', 'n_cp', 'sample', 'tstep'])

In [ ]:
ix1 = data.index

a = list(ix1.levels[0])
b = list(ix1.levels[1])
c = list(ix1.levels[2])

ix2 = pd.MultiIndex.from_product([a, b, c],
                                 names=('n_rn', 'n_cp', 'sample'))

In [ ]:
res1 = pd.DataFrame(index = ix2)

for i, j, k in tqdm_notebook(list(product(a, b, c))):
    
    tmp = data.xs(level=('n_rn', 'n_cp', 'sample'), key=(i, j, k)).reset_index()[['decision state', 'tstep', 'G']]
    t = None
    for _, row in tmp.iterrows():
        if row['decision state'] > .7 and t is None:
            t = row['tstep']
            G = row['G']
    if t is not None:
        if t < 5:
            t = None
    res1.loc[(i, j, k), 't'] = t
    res1.loc[(i, j, k), 'G'] = G
res1.unstack('sample')

In [ ]:
import seaborn as sn

g_max = data['G'].max()

plt_res = (res1['G']<g_max*(3./4.)).unstack('sample').sum(axis=1).unstack('n_cp')/50

sn.heatmap(plt_res)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,5))

plt_mean = (data.xs(level='tstep', key=100)['G'].unstack('sample')<g_max*(.75)).mean(axis=1).unstack('n_cp')
plt_std = (data.xs(level='tstep', key=100)['G'].unstack('sample')<g_max*(.75)).std(axis=1).unstack('n_cp')

sn.heatmap(plt_mean,
          ax=axes[0])

sn.heatmap(plt_std,
          ax=axes[1])

for ax, title in zip(axes, ['mean', 'standard deviation']):
    ax.set_xlabel('initial fraction of campaigners')
    ax.set_ylabel('initial fraction of rednecks')
    ax.set_title(title)

fig.savefig('prob_75pc_rec_left_in_2100.pdf')

In [ ]:
res2 = pd.DataFrame(index = ix2)

co2_per_oil = 3.5
initial_resource = data['G'].max()
budget = 886*10e2

for i, j, k in tqdm_notebook(list(product(a, b, c))):
    
    tmp = data.xs(level=('n_rn', 'n_cp', 'sample'), key=(i, j, k)).reset_index()[['decision state', 'tstep', 'G']]
    
    t = None
    for _, row in tmp.iterrows():
        budget_used = (initial_resource - row['G'])*co2_per_oil
        #print(row['tstep'], budget_used, budget)
        if budget_used > budget and t is None:
            t = row['tstep']
    if t is not None:
        if t < 5:
            t = None
    res1.loc[(i, j, k), 't'] = t
    res1.loc[(i, j, k), 'G'] = G
res1.unstack('sample')

In [ ]:
sn.heatmap(res1['t'].unstack('sample').std(axis=1).unstack('n_cp'))

In [ ]:
data = data.reset_index()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12,5))
data['G'].unstack(['n_rn', 'n_cp', 'sample']).plot(legend=False,
                                                  ax=axes[0],
                                                  color='k',
                                                  alpha=0.2)

In [ ]:
data['G'].max()*(.7)

In [ ]:
co2_per_oil = 3.5
initial_resource = data['G'].max()

In [ ]:
initial_resource